# 12: SalesOps Suite: Final Showcase 🎥

**"From Raw Data to Autonomous Action"**

This notebook visualizes the end-to-end journey of the SalesOps Agent.
It consumes the artifacts generated by our autonomous pipeline run.

### 📊 The Pipeline
1.  **Ingest:** 9,994 rows processed.
2.  **Detect:** Found statistical anomalies (Z-Score > 3.0).
3.  **Explain:** Gemini 2.0 analyzed root causes using RAG.
4.  **Act:** Created Jira tickets & Sent Emails.

## 1: Setup

In [1]:
import sys
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

# Add project root
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

os.environ["OBSERVABILITY_DIR"] = os.path.join(project_root, "outputs", "observability")

from dashboard.utils.loaders import (
    load_snapshot,
    load_anomalies,
    load_enriched,
    load_actions,
    get_kpis,
)

print("✅ Showcase Environment Ready")

2025-11-29 19:29:42.496 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:42.497 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:42.499 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:42.500 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:42.501 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


✅ Showcase Environment Ready


## 2: KPIs

In [2]:
# 1. High-Level Business Health
df = load_snapshot()
kpis = get_kpis(df)

# Create a beautiful indicator chart
fig = go.Figure()

fig.add_trace(
    go.Indicator(
        mode="number+delta",
        value=kpis["revenue"],
        title={"text": "Total Revenue"},
        number={"prefix": "$"},
        domain={"row": 0, "column": 0},
    )
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=kpis["profit"],
        title={"text": "Total Profit"},
        number={"prefix": "$"},
        domain={"row": 0, "column": 1},
    )
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=kpis["margin"],
        title={"text": "Profit Margin"},
        number={"suffix": "%", "valueformat": ".1f"},
        domain={"row": 0, "column": 2},
    )
)

fig.update_layout(grid={"rows": 1, "columns": 3, "pattern": "independent"}, height=200)
fig.show()

2025-11-29 19:29:42.508 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:42.510 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:42.799 
  command:

    streamlit run c:\Users\YASH\anaconda3\envs\salesops\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-11-29 19:29:42.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:42.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:42.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:42.924 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:42.924 Thread 'Main

## 3: The Problem - Anomalies

In [3]:
# 2. The Hidden Problems (Anomalies)
df_anoms = load_anomalies()

if not df_anoms.empty:
    # Filter for high severity
    top_anoms = df_anoms[df_anoms["score"] > 3.0].head(10)

    fig = px.scatter(
        top_anoms,
        x="period_start",
        y="score",
        size="score",
        color="detector",
        hover_data=["entity_id", "metric"],
        title="🚨 Critical Anomalies Detected (Z-Score > 3.0)",
        color_discrete_sequence=["#EF553B", "#636EFA"],
    )
    fig.show()

2025-11-29 19:29:44.100 No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:44.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.124 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


## 4: The Solution - AI Explanations 

In [4]:
# 3. The Intelligence (Gemini Explanations)
df_enriched = load_enriched()

if not df_enriched.empty:
    # Pick the most interesting one (Highest Score)
    best_rec = df_enriched.loc[df_enriched["score"].idxmax()]

    md = f"""
### 🤖 AI Insight Spotlight
**Anomaly:** {best_rec['entity_id']} ({best_rec['level']})

> **"{best_rec['explanation_short']}"**

* **Root Cause:** {best_rec['explanation_full']}
* **Confidence:** {best_rec['confidence']}
* **Action Proposed:** {best_rec['suggested_actions']}

*(Generated by {best_rec['model']} in {best_rec['latency']}ms)*
"""
    display(Markdown(md))

2025-11-29 19:29:44.332 No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:44.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



### 🤖 AI Insight Spotlight
**Anomaly:** South (region)

> **"South region sales significantly exceeded expectations, driven by a large, atypical order."**

* **Root Cause:** The sales value of 23,661.23 is drastically higher than the expected value of 1,159.38. This is supported by the deviation score of 24.21. While historical context shows anomalies can occur due to bulk orders (e.g., Acme Corp. in Technology), there is no specific historical event provided for the South region sales that directly explains this spike.
* **Confidence:** High
* **Action Proposed:** ['Investigate the source of the large sales value in the South region.', 'Verify if this represents a genuine large order or a data error.']

*(Generated by gemini-2.5-flash-lite in 1892ms)*


## 5: The Action - Execution Log

In [5]:
# 4. Closing the Loop (Actions Taken)
df_actions = load_actions()

if not df_actions.empty:
    # Status Breakdown
    fig = px.pie(
        df_actions, names="type", title="Autonomous Actions Executed", hole=0.4
    )
    fig.show()

    print("Recent Audit Log:")
    display(df_actions[["timestamp", "type", "status"]].tail(3))

2025-11-29 19:29:44.350 No runtime found, using MemoryCacheStorageManager
2025-11-29 19:29:44.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-29 19:29:44.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Recent Audit Log:


,timestamp,type,status
7,2025-11-29T13:57:44.633788+00:00,create_ticket,success
8,2025-11-29T13:57:46.674396+00:00,create_ticket,success
9,2025-11-29T13:57:48.719285+00:00,create_ticket,success


## 🏆 Mission Accomplished

We have demonstrated a fully autonomous SalesOps workflow.
* **Data:** Ingested & Monitored.
* **Insight:** Generated by AI with Memory.
* **Action:** Executed safely via APIs.
